<a href="https://colab.research.google.com/github/huisung/president_go_kr_petitions_scraping/blob/master/get_petition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초 소요

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [28]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (145 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [0]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [0]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        bs = BeautifulSoup(wd.page_source, 'lxml')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [31]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 300))
    print(get_petition(wd, 582349))
    wd.quit()

{'number': 300, 'progress': '청원종료', 'title': '대기업의 잘못된 인터넷 개인가입자 민원 처리방식과 위약금', 'like': 0, 'contents': '저는 SK인터넷,TV를 가입해서 사용하고 있는 개인 가입자 입니다.8/22일 휴가를 마치고 집으로 오니 인터넷,TV가 안되어서 SK 장애부서인 106번 으로전화하여 수리요청을 했습니다.상담원이 관할지역 기사님을 알아보고는 오늘은 기사님들이 스케줄이 안되어서 장애처리를 내일 해야 된다고 하였습니다.그래서 오늘 아이들 인터넷으로 숙제도해야되고 TV시청도 해야되니 오늘 늦더라도 고쳐달라고 말하니,조금뒤 기술부서 실장님이 전화를 주셔서 어떻게 해도 금일중으로는  기사님을 수배할수 없기 때문에 장애처리를 할수 없고 내일 오전10시즘에 처리 할수 있다고 말했습니다.A/S기사님들은 오전에 그날 A/S처리  스케줄을 다잡고 나가는데 그러면 소비자 들은 장애가 나면 무조건 다음날 A/S처리를 봤아야 되니까? 소비자 잘못이 아니라 SK장비때문에 장애가 났는데 이거는 SK에서 긴급 장애처리 기사를 더고용해서 당일 장애건도 그날A/S처리를 해줘야 되는거 아닙니까.그리고 SK잘못때문에 장애가 났는데 당일 처리를 못해주면 인터넷 해지를 요청하니 수십만원의 위약금을 물어야 된다고 하는데 이것은 잘못된 처방 아닙니까.자기들 장비때문에 장애가나서 사용못하고 그래서 수리요청하니 당일 처리가 힘들고,그러면 해지하려하니 위약금 내라고 하고,완전히 개인가입자는 봉입니까.그리고 처음 인터넷가입시 장애처리 문제에 대해서는 설명도 없었는데 장애가 나고나니 스케줄 잡아서 내일 A/S받든지 아님 나갈 사람이 없어방법이 없다.알아서 해라라는 식으로 답변을 듣기만 해야되나요.장애신고 하는 사람들은 지금 인터넷,TV가 안되어서 전화를 하는 사람들이지 미리 고장날것을 알고 전화해서 예약스케줄을 잡는 사람들이 아닙니다.SK인터넷 장애처리부서는 생각좀 하시고 일합시다.', 'answer': '', 'category': '기타', 's

## 사용예

In [34]:
import pandas as pd


def get_p(num = 583432, cnt = 10):
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    for idx, i in enumerate(tqdm(range(num-cnt, num))):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
%time rtn.append(list(get_petition(wd, 582349).values()))
%time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
pd.DataFrame(rtn, columns=c)


CPU times: user 6.8 ms, sys: 3.04 ms, total: 9.84 ms
Wall time: 1.2 s
CPU times: user 49.3 ms, sys: 0 ns, total: 49.3 ms
Wall time: 8.4 s
10 Message: 

CPU times: user 6.52 ms, sys: 879 µs, total: 7.4 ms
Wall time: 3.64 s


,number,progress,title,like,contents,answer,category,sday,eday,proponent
0,583422,청원종료,일산을 제발 살려주세요.,671,일산의 조정지역해제를 간곡히 부탁드립니다!\n김현미국토부장관님 문재인대통령님..\n...,,정치개혁,2019-11-04,2019-12-04,naver
1,583423,청원종료,"악덕건물주의 횡포와 사기,몇번의 양도방해로 인해 2년동안 통곡할억울함에 고통을 겪고...",188,제가 올리는 이내용이 세상에 널리 알려진다면 2년여동안 시달림을 당해왔던 건물주가족...,,기타,2019-11-04,2019-12-04,naver
2,583424,청원종료,****교회 법인허가 취소 해주세요.,715,고유번호 ***-**-***** ****교회는 민법에 의거해 설립 허가를 받은 비영...,,문화/예술/체육/언론,2019-11-04,2019-12-04,kakao
3,583425,청원종료,"공군 **전투비행단 군종실장 ***목사의 갑질 및 권력남용과 인권침해, 사적지시 사...",434,<직무태만>\n1. 본인의 주업무인 군종장교 및 군종실장 업무를 병사에게 지시\n(...,,인권/성평등,2019-11-04,2019-12-04,naver
4,583426,청원종료,음주운전으로 인한 근로자 사망,317,안녕하세요 저는 도로공사 교량보수공사를 하던 36살 근로자였습니다 5월10일경 중부...,,교통/건축/국토,2019-11-04,2019-12-04,naver
5,583427,청원종료,닭장같은 아파트 동간거리 9미터 못박은 서울시 갑질행정을 시정해주세요,699,한남 뉴타운의 현재 서울시의 설계 가이드라인이 박원순 사장 주도하에 공공건축가를 통...,,교통/건축/국토,2019-11-04,2019-12-04,naver
6,583428,청원종료,검찰피해자 조사위원회를 설치해 주세요,45180,"존경하는 대통령님!\n검찰피해자 조사위원회를 판,검사 출신이 아닌 외부인으로 구성하...",,정치개혁,2019-11-04,2019-12-04,facebook
7,583429,청원종료,"흑자기업 ****** 상장폐지, 이게 나라입니까?",687,"금번 거래소에서 발표한 우수한 기업, 흑자를 내고 있는 기업임에도 버젖이 상폐되어야...",,경제민주화,2019-11-04,2019-12-04,kakao
8,583430,청원종료,조국 전 장관 관련 ****** 상장폐지된다면 국민개미들 다 죽습니다.,639,안녕하십니까. 먼저 이렇게 이곳 청와대 청원장소까지 오게 되어 깊은 슬픔과 후회가 ...,,인권/성평등,2019-11-04,2019-12-04,naver
9,583431,청원종료,공영방송을 선거로 정할 수 있게 해주십시오.,527,KBS가 사용하고 있는 공영방송의 타이틀을 국민이 선거로 바꿀 수 있게 해주십시오....,,문화/예술/체육/언론,2019-11-04,2019-12-04,twitter


[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다